In [1]:
import json
import os
import shutil
from tqdm import tqdm
import yaml

In [2]:
LESSON = yaml.safe_load(open("STEP_0_define_lesson.yaml", "r"))['lesson']
print(LESSON)

lesson_13_at_the_restaurant


In [3]:
REPO_DIR = '/Users/stevie/repos/language_app'
assert(os.path.exists(REPO_DIR))

DEST_DIR = os.path.join('/Users/stevie/repos/language-app-frontend/assets/lessons', LESSON)
if not os.path.exists(DEST_DIR):
    os.mkdir(DEST_DIR)

In [4]:
json_path = os.path.join(REPO_DIR, f'language_notes/spotify_lesssons/{LESSON}/data.json')
data = json.load(open(json_path))

In [5]:
path_rel = data[0]['combined_audio_file']
assert(os.path.exists(os.path.join(REPO_DIR, path_rel)))
# print(os.path.join(DEST_DIR, path_rel))
print(path_rel)

language_notes/spotify_lesssons/lesson_13_at_the_restaurant/audio/0.mp3


In [6]:
for sample_i, sample_obj in tqdm(enumerate(data), total=len(data)):
    # combined audio file
    path = os.path.join(REPO_DIR, sample_obj['combined_audio_file'])
    assert(os.path.exists(path))
    rel_new_path = f'./audio/combined/{os.path.basename(path)}'
    new_path = os.path.join(DEST_DIR, rel_new_path)
    os.makedirs(os.path.dirname(new_path), exist_ok=True)
    if not os.path.exists(new_path):
        shutil.copyfile(path, new_path)
    path_str = f'require("{rel_new_path}"):::'
    data[sample_i]['combined_audio_file'] = path_str

    # individual audio files
    for language in 'english', 'italian':
        path = os.path.join(REPO_DIR, sample_obj[language]['audio_file'])
        assert(os.path.exists(path))
        rel_new_path = f'./audio/individual/{os.path.basename(path)}'
        new_path = os.path.join(DEST_DIR, rel_new_path)
        os.makedirs(os.path.dirname(new_path), exist_ok=True)
        if not os.path.exists(new_path):
            shutil.copyfile(path, new_path)
        path_str = f'require("{rel_new_path}"):::'
        data[sample_i][language]['audio_file'] = path_str

100%|██████████| 33/33 [00:00<00:00, 2305.45it/s]


In [7]:
js_path = os.path.join(REPO_DIR, f'language_notes/spotify_lesssons/{LESSON}/data.js')
json.dump(data, open(js_path, 'w'), indent=2, ensure_ascii=False)
text = open(js_path).read()
text = text.replace('\\"', '"')
text = f"samples = {text}"
text = text.replace('"require', 'require').replace(':::"', '')
text += ';\nexport default samples;'
open(js_path, 'w').write(text)

26748

In [8]:
old_js_path = os.path.join(REPO_DIR, f'language_notes/spotify_lesssons/{LESSON}/data.js')
new_js_path = os.path.join(DEST_DIR, 'data.js')
shutil.copyfile(old_js_path, new_js_path)

old_json_path = os.path.join(REPO_DIR, f'language_notes/spotify_lesssons/{LESSON}/data.json')
new_json_path = os.path.join(DEST_DIR, 'data.json')
shutil.copyfile(old_json_path, new_json_path)

old_csv_path = os.path.join(REPO_DIR, f'language_notes/spotify_lesssons/{LESSON}/transcript.csv')
new_csv_path = os.path.join(DEST_DIR, 'transcript.csv')
shutil.copyfile(old_csv_path, new_csv_path)

'/Users/stevie/repos/language-app-frontend/assets/lessons/lesson_13_at_the_restaurant/transcript.csv'